In [ ]:
from imutils.object_detection import non_max_suppression
from imutils import paths
import numpy as np
import argparse
import imutils
import cv2
import os
from scipy.spatial import distance as dist
from collections import OrderedDict

In [ ]:
class Group:
    def __init__(self):
        self.g = []
        self.centroid = None
        
        
    def add(self, object):
        self.g.append(object.tolist())
        self.update_centroid()

        
    def remove(self, object):
        self.g.remove(object.tolist())
        try:
            self.update_centroid()
        except:
            pass
        
    def getNum(self):
        return len(self.g)

    def has(self, o):
        if type(o) != list:
            o = o.tolist()
        if o in self.g:
            return True
        return False
    
    def update_centroid(self):
        sum_x=0
        sum_y=0
        for i in self.g:
            sum_x+=i[0]
            sum_y+=i[1]
        self.centroid = np.array([sum_x/len(self.g), sum_y/len(self.g)], dtype="int")
        
        
    def get_bound_centroid(self):
        left_most = sorted(self.g, key = lambda x: x[0])[0]
        right_most = sorted(self.g, key= lambda x:x[0])[-1]
        top_most = sorted(self.g, key=lambda x:x[1])[0]
        bot_most = sorted(self.g, key=lambda x:x[1])[-1]
        return top_most, left_most, right_most, bot_most
    

class TrackableObject:
    def __init__(self, objectID, centroid):
        # store the object ID, then initialize a list of centroids
        # using the current centroid
        self.objectID = objectID
        self.centroids = [centroid]

        # initialize a boolean used to indicate if the object has
        # already been counted or not
        self.counted = False

# implemented based on Adrain Rosebrock's algorithm 
# source https://www.pyimagesearch.com/2018/08/13/opencv-people-counter/
class CentroidTracker:
    def __init__(self, maxDisappeared=2, maxDistance=50):
        # initialize the next unique object ID along with two ordered
        # dictionaries used to keep track of mapping a given object
        # ID to its centroid and number of consecutive frames it has
        # been marked as "disappeared", respectively
        self.nextObjectID = 0
        self.objects = OrderedDict()
        self.disappeared = OrderedDict()

        # store the number of maximum consecutive frames a given
        # object is allowed to be marked as "disappeared" until we
        # need to deregister the object from tracking
        self.maxDisappeared = maxDisappeared

        # store the maximum distance between centroids to associate
        # an object -- if the distance is larger than this maximum
        # distance we'll start to mark the object as "disappeared"
        self.maxDistance = maxDistance
        
        
        self.groups = []
        self.group_distance = 100

    def register(self, centroid):
        # when registering an object we use the next available object
        # ID to store the centroid
        self.objects[self.nextObjectID] = centroid
        self.disappeared[self.nextObjectID] = 0
        self.nextObjectID += 1

    def deregister(self, objectID):
        # to deregister an object ID we delete the object ID from
        # both of our respective dictionaries
        g = self.getGroup(self.objects[objectID])
        if g != None:
            g.remove(self.objects[objectID])
            
        self.groups = [x for x in self.groups if x.getNum()>1]#update groups

        
        del self.objects[objectID]
        del self.disappeared[objectID]

    def update(self, rects):
        # check to see if the list of input bounding box rectangles
        # is empty
        if len(rects) == 0:
            # loop over any existing tracked objects and mark them
            # as disappeared
            for objectID in list(self.disappeared.keys()):
                self.disappeared[objectID] += 1

                # if we have reached a maximum number of consecutive
                # frames where a given object has been marked as
                # missing, deregister it
                if self.disappeared[objectID] > self.maxDisappeared:
                    self.deregister(objectID)

            # return early as there are no centroids or tracking info
            # to update
            return self.objects

        # initialize an array of input centroids for the current frame
        inputCentroids = np.zeros((len(rects), 2), dtype="int")

        # loop over the bounding box rectangles
        for (i, (startX, startY, endX, endY)) in enumerate(rects):
            # use the bounding box coordinates to derive the centroid
#             print("inside start", startX, startY)
#             print("inside end", endX, endY)

            cX = int((startX + (startX+ endX)) / 2.0)
            cY = int((startY + (startY+ endY)) / 2.0)
            inputCentroids[i] = (cX, cY)

        # if we are currently not tracking any objects take the input
        # centroids and register each of them
        if len(list(self.objects.values())) == 0:
            for i in range(0, len(inputCentroids)):
                self.register(inputCentroids[i])

        # otherwise, are are currently tracking objects so we need to
        # try to match the input centroids to existing object
        # centroids
        else:
            # grab the set of object IDs and corresponding centroids
            objectIDs = list(self.objects.keys())
            objectCentroids = list(self.objects.values())

            # compute the distance between each pair of object
            # centroids and input centroids, respectively -- our
            # goal will be to match an input centroid to an existing
            # object centroid
            D = dist.cdist(np.array(objectCentroids), inputCentroids)

            # in order to perform this matching we must (1) find the
            # smallest value in each row and then (2) sort the row
            # indexes based on their minimum values so that the row
            # with the smallest value as at the *front* of the index
            # list
            rows = D.min(axis=1).argsort()

            # next, we perform a similar process on the columns by
            # finding the smallest value in each column and then
            # sorting using the previously computed row index list
            cols = D.argmin(axis=1)[rows]

            # in order to determine if we need to update, register,
            # or deregister an object we need to keep track of which
            # of the rows and column indexes we have already examined
            usedRows = set()
            usedCols = set()

            # loop over the combination of the (row, column) index
            # tuples
            for (row, col) in zip(rows, cols):
                # if we have already examined either the row or
                # column value before, ignore it
                if row in usedRows or col in usedCols:
                    continue

                # if the distance between centroids is greater than
                # the maximum distance, do not associate the two
                # centroids to the same object
                if D[row, col] > self.maxDistance:
                    continue

                # otherwise, grab the object ID for the current row,
                # set its new centroid, and reset the disappeared
                # counter
                objectID = objectIDs[row]
                self.objects[objectID] = inputCentroids[col]
                self.disappeared[objectID] = 0

                # indicate that we have examined each of the row and
                # column indexes, respectively
                usedRows.add(row)
                usedCols.add(col)

            # compute both the row and column index we have NOT yet
            # examined
            unusedRows = set(range(0, D.shape[0])).difference(usedRows)
            unusedCols = set(range(0, D.shape[1])).difference(usedCols)

            # in the event that the number of object centroids is
            # equal or greater than the number of input centroids
            # we need to check and see if some of these objects have
            # potentially disappeared
            if D.shape[0] >= D.shape[1]:
                # loop over the unused row indexes
                for row in unusedRows:
                    # grab the object ID for the corresponding row
                    # index and increment the disappeared counter
                    objectID = objectIDs[row]
                    self.disappeared[objectID] += 1

                    # check to see if the number of consecutive
                    # frames the object has been marked "disappeared"
                    # for warrants deregistering the object
                    if self.disappeared[objectID] > self.maxDisappeared:
                        self.deregister(objectID)

            # otherwise, if the number of input centroids is greater
            # than the number of existing object centroids we need to
            # register each new input centroid as a trackable object
            else:
                for col in unusedCols:
                    self.register(inputCentroids[col])

        # return the set of trackable objects
        return self.objects
    
    
    def getGroup(self, obj):
        for group in self.groups:
            if group.has(obj):
                return group
            
        return None
    
    def d(self, a, b):
        return np.sqrt(np.sum(np.square(a-b)))
    
    def calculate_group(self):
        for i, person_i in enumerate(list(self.objects.values())):
            for j, person_j in enumerate(list(self.objects.values())):
                if j <= i:
                    continue
                    
                i_group = self.getGroup(person_i)
                j_group = self.getGroup(person_j)
                
                # if i in group
                if i_group != None:
                    #if they are in the same group. Check whether to ungroup if d(i_group, j) > group_distance
                    if i_group == j_group: 
                        distance = self.d(i_group.centroid, person_j)
                        # if distance > gourp_distance, ungroup
                        if distance > self.group_distance:
                            i_group.remove(person_j)
                    # they are not in the same group and i has group    
                    else: 
                        distance = self.d(i_group.centroid, person_j)
                        if distance < self.group_distance:
                            i_group.add(person_j)

                else:# if i not in group
                    if j_group == None: # if both not in group
                        distance = self.d(person_i, person_j)
                        if distance < self.group_distance:
                            g = Group()
                            g.add(person_i)
                            g.add(person_j)
                            self.groups.append(g)

                    else:
                        distance = self.d(person_i, j_group.centroid)
                        if distance < self.group_distance:
                            j_group.add(person_i)

                self.groups = [x for x in self.groups if x.getNum()>1]#update groups
                 
                    
                    
    def get_group_people_num(self):
        count=0
        for i in list(self.objects.values()):
            if self.getGroup(i) != None:
                count+=1
        return count
    


    def draw_group(self, image, label_colors, class_ids):
        width, height,_ = image.shape
        for i in list(self.objects.values()):
            cv2.circle(image, (i[0], i[1]), 5, (0,255,0),2)
        for group in self.groups:
            x = group.centroid[0]
            y = group.centroid[1]
            color = int(label_colors[class_ids[-1]][0])
#             cv2.rectangle(image, (x-self.group_distance, y-self.group_distance), (x + self.group_distance, y + self.group_distance), color, 2)
            cv2.circle(image, (x, y), self.group_distance, color, 2)
        self.groups=[]

        

In [ ]:
def draw_detection(image,boxes, indexes, label_colors, class_ids):
    for i, _ in enumerate(boxes):
        x, y = (boxes[i][0], boxes[i][1])
        w, h = (boxes[i][2], boxes[i][3])
        color = [int(c) for c in label_colors[class_ids[i]]]
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)


def detect_person(image):

    height, width, _ = image.shape

    blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    layer_outputs = net.forward(layer_names)

    boxes = []
    confidences = []
    class_ids = []
    center_boxes = []

    for output in layer_outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            if labels[class_id] != "person":
                continue

            confidence = scores[class_id]
            if confidence > confidence_threshold:
                box = detection[0:4] * np.array([width, height, width, height])
                center_x, center_y, box_width, box_height = box.astype("int")
                x = int(center_x - box_width / 2)
                y = int(center_y - box_height / 2)
                boxes.append([x, y, int(box_width), int(box_height)])
                center_boxes.append([center_x, center_y])
                confidences.append(float(confidence))
                class_ids.append(class_id)
                
    return boxes, confidences, class_ids



confidence_threshold = 0.5
nms_threshold = 0.4  


labels = []
with open("coco.names", 'rt') as f:
    labels = f.read().rstrip('\n').split('\n')

np.random.seed(42)
label_colors = np.random.randint(0, 255, size=(len(labels), 3), dtype="uint8")

net = cv2.dnn.readNetFromDarknet("yolo.cfg", "yolov3.weights")
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

layer_names = net.getLayerNames()
layer_names = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

tracker = CentroidTracker()


image_sequence_path = "sequence"
output_video_path = "task3_GD=100.avi"

cap = cv2.VideoCapture(f"{image_sequence_path}/%06d.jpg", cv2.CAP_IMAGES)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_video_path, fourcc, 10.0,
                      (frame_width * 2, frame_height * 2))

image_id = 1
while (cap.isOpened()):
    ret, frame = cap.read()
    if not ret:
        break

    if image_id %100 == 0:
        print(image_id)

    resized_frame = cv2.resize(frame, (frame_width * 2, frame_height * 2))

    boxes, confidences, class_ids = detect_person(resized_frame)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, nms_threshold)
    
    new_boxes = []

    for i in indexes.flatten():
        new_boxes.append(boxes[i])
        
    tracker.update(new_boxes)
    draw_detection(resized_frame, new_boxes, indexes, label_colors, class_ids)

    tracker.calculate_group()
    
    group_num = tracker.get_group_people_num()
    individual = len(tracker.objects) - group_num
    
    tracker.draw_group(resized_frame, label_colors, class_ids)


    cv2.putText(resized_frame,"sequence id: {}".format(image_id), (20, frame_height*2 - 100),
                cv2.FONT_HERSHEY_SIMPLEX,
                1, (255, 255, 255), 3)
    cv2.putText(resized_frame, "People in individual: {}".format(individual), (20, frame_height*2 - 20),
                cv2.FONT_HERSHEY_SIMPLEX,
                1, (255,255,255), 3)
    cv2.putText(resized_frame,"People in group: {}".format(group_num), (20, frame_height*2 - 60),
                cv2.FONT_HERSHEY_SIMPLEX,
                1, (255, 255, 255), 3)
    cv2.putText(resized_frame, "People in individual: {}".format(individual), (20, frame_height*2 - 20),
                cv2.FONT_HERSHEY_SIMPLEX,
                1, (255,255,255), 3)
    out.write(resized_frame)
    


    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    image_id +=1
    

cap.release()
out.release()
cv2.destroyAllWindows()

